# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [23]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
total=sinasc.shape[0]
sem_dupl=sinasc.drop_duplicates().shape[0]
print('total: {0}, sem duplicados: {1}'.format(total,sem_dupl))
if(total==sem_dupl):
    print('Não há duplicados')
else:
    print('Há duplicados')

(27028, 69)
total: 27028, sem duplicados: 27028
Não há duplicados


In [14]:
# 2) seu código aqui
total_miss=sinasc.isna().sum()
print('total de missing: {0}'.format(total_miss))


total de missing: ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


In [22]:
# 3) seu código aqui
df_selecao=sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
total_sel=df_selecao.shape[0]
sem_dupl_sel=df_selecao.drop_duplicates().shape[0]
print('total: {0}, sem duplicados: {1}'.format(total_sel,sem_dupl_sel))
if(total_sel==sem_dupl_sel):
    print('Não há duplicados')
else:
    print('Há duplicados')

total: 27028, sem duplicados: 10058
Há duplicados


In [25]:
# 4) seu código aqui
df_apgar=df_selecao[~df_selecao['APGAR5'].isna()]
total_apgar=df_apgar.shape[0]
sem_dupl_apgar=df_apgar.drop_duplicates().shape[0]
print('total: {0}, sem duplicados: {1}'.format(total_apgar,sem_dupl_apgar))
if(total_apgar==sem_dupl_apgar):
    print('Não há duplicados')
else:
    print('Há duplicados')

total: 26925, sem duplicados: 9956
Há duplicados


In [48]:
# 5) seu código aqui
df_ignorados=df_apgar[(~df_apgar['ESTCIVMAE'].isna())|(~df_apgar['CONSULTAS'].isna())]
df_ignorados=df_ignorados[(df_ignorados['CONSULTAS']!=9)&(df_ignorados['ESTCIVMAE']!=9.0)]
df_ignorados

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [52]:
# 6) Seu código aqui
df_ignorados['QTDFILVIVO'].fillna(0,inplace=True)
df_ignorados

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [71]:
# 7) seu código aqui
#['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
df_ignorados['LOCNASC'].fillna(9,inplace=True) #Local Nascimento será 9: Ignorado
med_idademae = df_ignorados['IDADEMAE'].mean()
df_ignorados['IDADEMAE'].fillna(med_idademae,inplace=True)# idade da mae será preenchido pela média
df_ignorados['ESCMAE'].fillna('Nenhuma',inplace=True)# escolaridade da mae será preenchido por Nenhuma
df_ignorados['QTDFILVIVO'].fillna(0.0,inplace=True)# preencher  com 0.0
df_ignorados['GESTACAO'].fillna('Menos de 22 semanas',inplace=True)
df_ignorados['GRAVIDEZ'].fillna('Única',inplace=True)
df_ignorados['CONSULTAS'].fillna(1,inplace=True)
df_ignorados


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [74]:
# 8) seu código aqui
'''
Entre 8 e 10 está em uma faixa 'normal'.
Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
Entre 4 e 5 significa 'asfixia moderada'.
Entre 0 e 3 significa 'asfixia severa'.
'''
df_ignorados.loc[(df_ignorados['APGAR5'] <= 10) & (df_ignorados['APGAR5'] >= 8), 'class_asfixia'] = 'normal'
df_ignorados.loc[(df_ignorados['APGAR5'] <= 7) & (df_ignorados['APGAR5'] >= 6), 'class_asfixia'] = 'asfixia leve'
df_ignorados.loc[(df_ignorados['APGAR5'] <= 5) & (df_ignorados['APGAR5'] >= 4), 'class_asfixia'] = 'asfixia moderada'
df_ignorados.loc[(df_ignorados['APGAR5'] <= 3) & (df_ignorados['APGAR5'] >= 0), 'class_asfixia'] = 'asfixia severa'
df_ignorados['class_asfixia'].value_counts()

normal              26272
asfixia leve          318
asfixia severa         73
asfixia moderada       68
Name: class_asfixia, dtype: int64

In [76]:
# 9) seu código aqui
df_ignorados.rename(columns = {'LOCNASC':'LOC_NASC','IDADEMAE':'IDADE_MAE','ESTCIVMAE':'EST_CIV_MAE',
                              'ESCMAE':'ESC_MAE','QTDFILVIVO':'QTD_FIL_VIVO',
                              'class_asfixia':'CLASS_ASFIXIA'}, inplace = True)
df_ignorados

,LOC_NASC,IDADE_MAE,EST_CIV_MAE,ESC_MAE,QTD_FIL_VIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,CLASS_ASFIXIA
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal
